In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 6

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))

In [4]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        
        inputs = inputs.to(device)
        labels = labels.to(device)
                

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.781
[1,  4000] loss: 1.527
[1,  6000] loss: 1.433
[1,  8000] loss: 1.347
[2,  2000] loss: 1.283
[2,  4000] loss: 1.253
[2,  6000] loss: 1.218
[2,  8000] loss: 1.217
[3,  2000] loss: 1.143
[3,  4000] loss: 1.159
[3,  6000] loss: 1.131
[3,  8000] loss: 1.146
[4,  2000] loss: 1.081
[4,  4000] loss: 1.066
[4,  6000] loss: 1.089
[4,  8000] loss: 1.074
[5,  2000] loss: 1.020
[5,  4000] loss: 1.055
[5,  6000] loss: 1.041
[5,  8000] loss: 1.026
[6,  2000] loss: 0.982
[6,  4000] loss: 0.993
[6,  6000] loss: 0.989
[6,  8000] loss: 0.995
[7,  2000] loss: 0.931
[7,  4000] loss: 0.949
[7,  6000] loss: 0.989
[7,  8000] loss: 0.977
[8,  2000] loss: 0.909
[8,  4000] loss: 0.919
[8,  6000] loss: 0.941
[8,  8000] loss: 0.964
[9,  2000] loss: 0.885
[9,  4000] loss: 0.896
[9,  6000] loss: 0.923
[9,  8000] loss: 0.926
[10,  2000] loss: 0.857
[10,  4000] loss: 0.867
[10,  6000] loss: 0.901
[10,  8000] loss: 0.907
[11,  2000] loss: 0.815
[11,  4000] loss: 0.862
[11,  6000] loss: 0.869
[11,

In [5]:
total = 0 
correct =0

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 60 %


In [6]:
# batch size - 4; 
# test accuracy - 54%
# number of epochs - 2
# lr=0.001
# momentum=0.9
# optimizer - Adam 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))